# TensorFlow Introduction

In [1]:
import tensorflow as tf
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

# sess = tf.Session()
# sess.run(x.initializer)
# sess.run(y.initializer)
# result = sess.run(f)
# print(result)
# sess.close()

init = tf.global_variables_initializer()
# sess = tf.InteractiveSession() Jupyter の場合はこれ使うといいかもらしい

with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


In [2]:
w = tf.constant(3) 
x=w+2 
y=x+5 
z=x*3

with tf.Session() as sess:
    print(y.eval())  # 10
    print(z.eval())  # 15

# こうしたら1回だけ評価
# with tf.Session() as sess:
#     y_val, z_val = sess.run([y, z])
#     print(y_val)  # 10
#     print(z_val)  # 15

10
15


TensorFlow は自動的に y が x に依存し、x が w に依存することに気付き、 まず w、次に x、さらにその次に y を評価して、y の値を返す。最後に、コードは z を評価するグラ フを実行する。TensorFlow は、ここでまず w と x を評価しなければならないことに気付く。ここで大切なのは、w と x の前の評価結果を再利用しないことだ。つまり、上のコードは w と x を 2 回 評価している。

シンプルプロセス TensorFlow と分散 TensorFlow とは？

In [3]:
import numpy as np
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y) # theta は正規方程式 (4章らしい)
with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

[[-3.6894890e+01]
 [ 4.3661433e-01]
 [ 9.4453208e-03]
 [-1.0704148e-01]
 [ 6.4345831e-01]
 [-3.9632569e-06]
 [-3.7880042e-03]
 [-4.2093179e-01]
 [-4.3400639e-01]]


上のコードは 2 次元配列を操作してカリフォル ニアの住宅価格データセット(2 章参照)に対する線形回帰を行う。このコードはまずデータセッ トをフェッチし、すべての訓練インスタンスにバイアス入力特徴量(x0 = 1)を追加する(すぐに挿入を行う NumPy を使っている)。次に、このデータとターゲットを格納する X と y のふたつ の TensorFlow 定数ノードを作る。そして、TensorFlow の行列演算を使って theta を定義する。

### 9.6.1 

- random_uniform() 関数は、NumPy の rand() 関数と同じように、与えられた形状と 値の範囲に基づいて無作為値を格納するテンソルを生成する。
- assign() 関数は、変数に新しい値を代入するノードを作る。この場合は、バッチ勾配降 下法のステップ θ(next step) = θ − η∇θ MSE(θ) を実装している。
- メインループは訓練ステップを何度も繰り返し(n_epochs 回)、100 回反復するたびに、 その時点での平均二乗誤差(mse)を表示する。MSE がだんだん小さくなるのがわかる。

In [17]:
n_epochs = 1000
learning_rate = 0.01
# X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")

y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
# gradients = 2/m * tf.matmul(tf.transpose(X), error)
# gradients = tf.gradients(mse, [theta])[0]

# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

training_op = tf.assign(theta, theta - learning_rate * gradients)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        sess.run(training_op)
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            
        best_theta = theta.eval()

Epoch 0 MSE = 4588443000000000.0
Epoch 100 MSE = 4.680679e+19
Epoch 200 MSE = 1.853788e+20
Epoch 300 MSE = 4.1571806e+20
Epoch 400 MSE = 7.3782416e+20
Epoch 500 MSE = 1.1517039e+21
Epoch 600 MSE = 1.6573614e+21
Epoch 700 MSE = 2.2547909e+21
Epoch 800 MSE = 2.9439932e+21
Epoch 900 MSE = 3.7249675e+21


gradients() 関数は、入力としてオペレーション(この場合は mse)と変数リスト(この場 合は theta)を与えられると、ops(変数ごとにひとつずつ)のリストを作り、個々の変数に対応 する勾配を計算する。そのため、gradients ノードは、theta について MSE の勾配ベクトル を計算することになる。`どうでもいい`

## 訓練アルゴリズムへのデータ供給

ミニバッチ勾配降下法を実装するように前のコードを書き換えてみよう。たとえば、イテレー ションごとに X と y を次のミニバッチに入れ替えていかなければならないものとする。もっとも 簡単な方法は、プレースホルダーノードを使うものである。プレースホルダーノードは、実際には 計算を行わず、実行時に出力せよと指示したデータを出力するだけだという点で特殊なノードであ る。一般に、訓練中に TensorFlow に訓練データを渡すために使われる。実行時にプレースホル ダーのための値を指定しなければ、例外が起きる。プレースホルダーノードを作るためには、出力テンソルのデータ型を指定して placeholder() を呼び出さなければならない。形状を強制したい場合は、オプションで形状も指定できる。次数と して None を指定した場合、「任意のサイズ」という意味になる。たとえば、次のコードは、A と いうプレースホルダーノードを作り、B = A + 5というノードも作る。Bを評価するときにAの 値を指定する feed_dict を eval() に渡す。A はランク 2 でなければならず(つまり 2 次元でなければならない)、3 つの列がなければならない(そうでなければ例外が生成される)。しかし、行数はいくつでもよい。

In [18]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})
    print(B_val_1)
    print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


ミニバッチ勾配降下法は、既存のコードを少し書き換えるだけで実装できる。まず、構築フェー ズの X と y を書き換え、プレースホルダーノードを作らせるようにする。

次に、バッチサイズを定義し、バッチの総数を計算する。

最後に、実行フェーズでミニバッチをひとつずつ取り出し、X か y に依存するノードを評価する ときに、feed_dict パラメータを介して X、y の値を与える